In [0]:
import numpy as np
import pandas as pd
import scipy.stats as sc
import scipy.optimize as optim
import scipy.linalg as sclinalg
import math

In [0]:
def create_G(dist,lda,xi,psi,T):
  #This function creates G_t with the desired distribution and sample sizes
  sample_size = int(T)

  if (dist == "MALap"):
    G_dist = sc.laplace(scale=lda)
    G = G_dist.rvs(size=sample_size)
  elif (dist == "NIG"):
    G_dist = sc.geninvgauss(p=lda, b=np.sqrt(xi*psi), scale=np.sqrt(xi/psi))
    G = G_dist.rvs(size=sample_size)
  elif (dist == "MAt"):
    G_dist = sc.t(df=xi)
    G = G_dist.rvs(size=sample_size)

  return G

In [0]:
def create_S(omega,alpha,beta,mu,gamma,T,K,y,G):
  #This function creates the matrix for S_t, which is governed by the garch(1,1) alike dynamics

  omega_tmp = omega.reshape(K,1)
  alpha_tmp = alpha.reshape(K,1)
  beta_tmp = beta.reshape(K,1)
  mu_tmp = mu.reshape(K,1)
  gamma_tmp = gamma.reshape(K,1)

  S_matrix = np.zeros([T+1,K,K])
  S_2 = np.divide(omega_tmp,1 - alpha_tmp - beta_tmp).reshape(K,1)
  S_matrix[0,:,:] = np.diag(np.sqrt(S_2).reshape(1,K).squeeze())

  for t in range(1,T+1):
    S_tmp = np.diag(S_matrix[t-1,:,:]).reshape(K,1)
    epsilon = y[:,t-1].reshape(K,1) - mu_tmp - gamma_tmp*G[t-1]
    S_2 = omega_tmp + np.multiply(alpha_tmp,np.square(epsilon)) + np.multiply(beta_tmp,np.square(S_tmp))

    #Checking purpose#
    if (S_2.any() < 0):
      print("Error!!! something wrong with omega,alpha,beta") #Checking purpose#

    S_matrix[t,:,:] = np.diag(np.sqrt(S_2).reshape(1,K).squeeze())

  return S_matrix[1:(T+1),:,:]

In [0]:
def log_likelihood_mean_vol(omega,alpha,beta,mu,gamma,T,K,y,G):
  #This function calculates the mean_vol part of the log-likelihood

  mu_tmp = mu.reshape(K,1)
  gamma_tmp = gamma.reshape(K,1)
  S = create_S(omega,alpha,beta,mu,gamma,T,K,y,G)
  
  sum = 0
  for t in range(T):
    term_1 = K*np.log(2*math.pi)

    term_2 = np.log(np.square(np.linalg.det(S[t,:,:])))

    epsilon = y[:,t].reshape(K,1) - mu_tmp - gamma_tmp*G[t]
    S_inv_2 = np.dot(np.linalg.pinv(S[t,:,:]),np.linalg.pinv(S[t,:,:]))
    term_3 = np.power(G[t],-1) * np.dot(np.dot(epsilon.T,S_inv_2),epsilon)

    #iter_sum = term_1 + term_2 + term_3 + np.log(np.power(G[t],K))
    iter_sum = term_1 + term_2 + term_3
    sum += iter_sum.item()
  
  return -0.5*sum

In [0]:
def log_likelihood_Corr(omega,alpha,beta,mu,gamma,Corr,T,K,y,G):
  #This function calculates the Corr part of the log-likelihood

  mu_tmp = mu.reshape(K,1)
  gamma_tmp = gamma.reshape(K,1)
  S = create_S(omega,alpha,beta,mu,gamma,T,K,y,G)
  
  sum = 0
  for t in range(T):
    epsilon = y[:,t].reshape(K,1) - mu_tmp - gamma_tmp*G[t]
    e_t = np.power(G[t],-0.5) * np.dot(np.linalg.pinv(S[t,:,:]),epsilon)

    term_1 = np.log(np.absolute(np.linalg.det(Corr)))
    term_2 = np.dot(np.dot(e_t.T,np.linalg.pinv(Corr)),e_t)
    term_3 = np.dot(e_t.T,e_t)

    iter_sum = term_1 + term_2 + term_3
    sum += iter_sum.item()
  
  return -0.5*sum

In [0]:
def log_likelihood_G(dist,lda,xi,psi,G):
  #This function calculates the log-likelihood of G_t

  if (dist == "MALap"):
    G_dist = sc.laplace(scale=lda)
  elif (dist == "NIG"):
    G_dist = sc.geninvgauss(p=lda, b=np.sqrt(xi*psi), scale=np.sqrt(xi/psi))
  elif (dist == "MAt"):
    G_dist = sc.t(df=xi)

  sum = 0
  for t in range(len(G)):
    sum += G_dist.logpdf(G[t]).item()
  
  return -0.5*sum  

In [0]:
def log_likelihood_complete(omega,alpha,beta,mu,gamma,Corr,dist,lda,xi,psi,T,K,y,G):
  #This function calculates the complete log-likelihood

  return log_likelihood_mean_vol(omega,alpha,beta,mu,gamma,T,K,y,G) + log_likelihood_Corr(omega,alpha,beta,mu,gamma,Corr,T,K,y,G) + log_likelihood_G(dist,lda,xi,psi,G)

In [0]:
def CM_1_P_para_reshape(omega,alpha,beta,mu,gamma,vec,k):
  #This function updates a row of values for all 5 parameter vectors that belong to the set Theta_P

  vec = vec.reshape(5,)
  omega_tmp = omega
  alpha_tmp = alpha
  beta_tmp = beta
  mu_tmp = mu
  gamma_tmp = gamma

  omega_tmp[k] = vec[0]
  alpha_tmp[k] = vec[1]
  beta_tmp[k] = vec[2]
  mu_tmp[k] = vec[3]
  gamma_tmp[k] = vec[4]
  return omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp

In [0]:
def flatten_Corr(Corr):
  #This function flattens the upper-triangular Correlation matrix, so the resulting vector could be passed to the optimization algo

  vec = []
  N = Corr.shape[0]
  for i in range(N-1):
    for j in range(i+1,N):
      vec.append(Corr[i,j])
  return vec

def CM_1_C_para_reshape(vec):
  #This function updates the Correlation matrix (set Theta_C)

  size = int(0.5 * (1 + np.sqrt(1 + 8*len(vec))))
  Corr_new = np.eye(size,size)
  
  tmp = 0
  for i in range(size-1):
    for j in range(i+1,size):
      Corr_new[i,j] = vec[tmp]
      tmp += 1

  i_lower = np.tril_indices(size, -1)
  Corr_new[i_lower] = Corr_new.T[i_lower]
  return Corr_new

In [0]:
def imputed_G(omega,alpha,beta,mu,gamma,Corr,dist,lda,xi,psi,T,K,y):
  #This function creates the imputed G_t for the EM algo

  #Calculate the matrix for s using the current set of parameters first
  G_old = create_G(dist,lda,xi,psi,T)
  S = create_S(omega,alpha,beta,mu,gamma,T,K,y,G_old)
  mu_tmp = mu.reshape(K,1)
  G_new = np.zeros([T,])

  #Set the Theta_D set for the imputed g
  if (dist == "MALap"):
    lda_new = np.maximum(lda - 0.5*K,10e-10)
    lda_new = lda_new.item()
    G_new = create_G(dist,lda_new,xi,psi,T)
  elif (dist == "NIG"):
    for t in range(T):
      H_t_inv = np.linalg.pinv(np.dot(np.dot(S[t,:,:],Corr),S[t,:,:]))
      adjust_term = np.dot(np.dot((y[:,t].reshape(K,1) - mu_tmp).T,H_t_inv),(y[:,t].reshape(K,1) - mu_tmp))
      xi_new = xi + adjust_term
      xi_new = xi_new.item()
      G_new[t] = create_G(dist,lda,xi_new,psi,1)
  elif (dist == "MAt"):
    lda_new = lda - 0.5*K
    lda_new = lda_new.item()
    for t in range(T):
      H_t_inv = np.linalg.pinv(np.dot(np.dot(S[t,:,:],Corr),S[t,:,:]))
      adjust_term = np.dot(np.dot((y[:,t].reshape(K,1) - mu_tmp).T,H_t_inv),(y[:,t].reshape(K,1) - mu_tmp))
      xi_new = xi + adjust_term
      xi_new = xi_new.item()
      G_new[t] = create_G(dist,lda_new,xi_new,psi,1)
  
  return G_new

In [0]:
def EM_algo(omega,alpha,beta,mu,gamma,Corr,dist,lda,xi,psi,T,K,y,iteration,optim_method,tolerance):
  #Main EM algorithm to estimate all parameters (Theta_P,Theta_C,Theta_D)
  diff = 1000
  current_it = 0
  E = []

  omega_tmp = omega
  alpha_tmp = alpha
  beta_tmp = beta
  mu_tmp = mu
  gamma_tmp = gamma
  Corr_tmp = Corr
  lda_tmp = lda
  xi_tmp = xi
  psi_tmp = psi

  while (current_it < iteration) and (diff > tolerance):
    current_it += 1 #Keep track of the current iteration of EM algo

    ######E-step#####
  
    ##Calculate the imputed g
    G_hat = imputed_G(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,Corr_tmp,dist,lda_tmp,xi_tmp,psi_tmp,T,K,y)
    print("G_hat done\n {}\n".format(G_hat))

    ##Calculate the expected value of complete log-likelihood
    E_mean_vol = log_likelihood_mean_vol(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,T,K,y,G_hat)
    E_Corr = log_likelihood_Corr(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,Corr_tmp,T,K,y,G_hat)
    E_G = log_likelihood_G(dist,lda_tmp,xi_tmp,psi_tmp,G_hat)
    E_complete_log_lld = E_mean_vol + E_Corr + E_G
    E.append(E_complete_log_lld)
    
    print("E-step done")
    print(E_complete_log_lld)#Checking purpose

    ######M-step#####

    ##CM1 (P) Updating Theta_P set

    #We individually estimate the Theta_P set for each asset
    for k in range(K):
      #First we create a lambda function to facilitate the optimization algo
      #The only input will be the parameter vector that we need to estimate
      CM_1_P_para_reshape_k = lambda vec: CM_1_P_para_reshape(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,vec,k)

      #Redefine the function that calculates the mean-vol part of the log-likelihood, so it only accepts a single vector as input
      def CM_1_P_func(x): 
        omega_new,alpha_new,beta_new,mu_new,gamma_new = CM_1_P_para_reshape_k(x)
        return -log_likelihood_mean_vol(omega_new,alpha_new,beta_new,mu_new,gamma_new,T,K,y,G_hat)

      #Initial value for the parameters (which is set as the value at last iteration)
      CM_1_P_init = np.array([omega_tmp[k], alpha_tmp[k], beta_tmp[k], mu_tmp[k], gamma_tmp[k]])
      
      #set bounds of the parameters
      bnds = ((10e-7, None), (10e-7, None),(10e-7, None),(None, None),(None, None))
      
      #Conduct Minimization
      CM_1_P = optim.minimize(fun=CM_1_P_func,x0=CM_1_P_init, method=optim_method, bounds=bnds)
      omega_tmp[k],alpha_tmp[k],beta_tmp[k],mu_tmp[k],gamma_tmp[k] = CM_1_P.x
      print("CM1(P)-step {} done".format(k+1))

    print("CM1(P)-step all done")
    #print(np.array([omega,alpha,beta,mu,gamma])) #Checking purpose


    ##CM1 (C) Updating Theta_C set

    #Redefine the function that calculates the Corr part of log-likelihood, so it only accepts a single vector as input
    def CM_1_C_func(x):
      Corr_new = CM_1_C_para_reshape(x)
      return -log_likelihood_Corr(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,Corr_new,T,K,y,G_hat)

    #Initial value for the optimization, done by flattening the upper-triangular Correlation matrix
    CM_1_C_init = flatten_Corr(Corr_tmp)
    
    #Set bounds for all values in the upper-triangular Correlation matrix
    bnds = []
    for i in range(len(CM_1_C_init)):
      bnds.append((-1,1))

    #Confuct Minimization
    CM_1_C = optim.minimize(fun=CM_1_C_func,x0=CM_1_C_init, method=optim_method,bounds=bnds)
    Corr_tmp = CM_1_C_para_reshape(CM_1_C.x)
    
    print("CM1(C)-step done")
    print(Corr_tmp) #Checking purpose

    
    ##CM2 Updating Theta_D set
    #For each distribution, the optimization parameter will be different, so we seperate into 3 cases

    if (dist == "MALap"):
      #Redefine the function that calculates the complete log-likelihood, so it only accepts a single number as input
      def CM_2_func_MALap(x):
        E_mean_vol = log_likelihood_mean_vol(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,T,K,y,G_hat)
        E_Corr = log_likelihood_Corr(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,Corr_tmp,T,K,y,G_hat)
        E_G = log_likelihood_G(dist,x,xi_tmp,psi_tmp,G_hat)
        return E_mean_vol + E_Corr + E_G

      #Set bounds for parameter
      bnds = ((10e-10,None),)

      #Conduct Minimization
      CM_2 = optim.minimize(fun=CM_2_func_MALap, x0=lda_tmp, method=optim_method, bounds=bnds)
      lda_tmp = CM_2.x

    elif (dist == "NIG"):
      #Redefine the function that calculates the complete log-likelihood, so it only accepts a single number as input
      def CM_2_func_NIG(x):
        E_mean_vol = log_likelihood_mean_vol(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,T,K,y,G_hat)
        E_Corr = log_likelihood_Corr(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,Corr_tmp,T,K,y,G_hat)
        E_G = log_likelihood_G(dist,lda_tmp,x,psi_tmp,G_hat)
        return E_mean_vol + E_Corr + E_G         
        
      #Set bounds for parameter
      bnds = ((10e-10,10),)

      #Conduct Minimization
      CM_2 = optim.minimize(fun=CM_2_func_NIG,x0=xi_tmp, method=optim_method, bounds=bnds)
      xi_tmp = (CM_2.x).item()

    elif (dist == "MAt"):
      #Redefine the function that calculates the complete log-likelihood, so it only accepts a single number as input
      def CM_2_func_MAt(x):
        E_mean_vol = log_likelihood_mean_vol(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,T,K,y,G_hat)
        E_Corr = log_likelihood_Corr(omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,Corr_tmp,T,K,y,G_hat)
        E_G = log_likelihood_G(dist,lda_tmp,x,psi_tmp,G_hat)
        return E_mean_vol + E_Corr + E_G         
        
      #Set bounds for parameter
      bnds = ((10e-10,None),)

      #Conduct Minimization
      CM_2 = optim.minimize(fun=CM_2_func_MAt,x0=xi_tmp, method=optim_method)
      xi_tmp = CM_2.x
      lda_tmp = -0.5*xi_tmp
    
    print("CM2-step done")
    print(np.array([lda_tmp,xi_tmp,psi_tmp])) #Checking purpose

    if (current_it > 1):
      diff = abs(E[current_it-1] - E[current_it-2])
      print("diff in iteration {a} is {b}".format(a=current_it,b=diff))
    
    print("Iteration {} completed".format(current_it))

  return omega_tmp,alpha_tmp,beta_tmp,mu_tmp,gamma_tmp,Corr_tmp,lda_tmp,xi_tmp,psi_tmp,E

In [0]:
def create_y(omega,alpha,beta,mu,gamma,Corr,T,K,G,Z):
  #This function creates simulated return y for checking purpose

  omega_tmp = omega.reshape(K,1)
  alpha_tmp = alpha.reshape(K,1)
  beta_tmp = beta.reshape(K,1)
  mu_tmp = mu.reshape(K,1)
  gamma_tmp = gamma.reshape(K,1)

  y = np.zeros([K,T])
  #S_2 = np.divide(omega_tmp,1 - alpha_tmp - beta_tmp).reshape(K,1)
  S_2 = np.zeros([K,1])
  epsilon = np.zeros([K,1])

  for t in range(T):
    print("iteration {}\n".format(t+1))

    S_2 = omega_tmp + np.multiply(alpha_tmp,np.square(epsilon)) + np.multiply(beta_tmp,S_2)
    print("S_2 is\n {}\n".format(S_2))

    #Checking purpose#
    if (S_2.any() < 0):
      print("Error!!! something wrong with omega,alpha,beta") #Checking purpose#

    S_reshape = np.diag(np.sqrt(S_2).reshape(1,K).squeeze())

    H_t = np.dot(np.dot(S_reshape,Corr),S_reshape)
    print("H_t is\n {}\n".format(H_t))

    H_t_sqrt = sclinalg.sqrtm(H_t)
    print("H_t_0.5 is\n {}\n".format(H_t_sqrt))

    Z = sc.norm.rvs(size=K)
    print("Z is\n {}\n".format(Z))

    if (G[t] < 0):
      print("Something wrong with G\n")
      print("G is {}\n".format(G[t]))
    else:
      G_sqrt = np.sqrt(G[t])
      print("sqrt G is {}\n".format(G_sqrt))

    epsilon = (G_sqrt * np.dot(H_t_sqrt,Z)).reshape(K,1)
    print("epsilon is\n {}\n".format(epsilon))
    
    y[:,t] = (mu_tmp + G[t] * gamma_tmp + epsilon).reshape(K,)
    print("result is\n {}\n".format(mu_tmp + G[t] * gamma_tmp + epsilon))

  return y

In [0]:
##Simulate test data##

#Set random seed
np.random.seed(1234)

#Initialization of parameters sets#
T_test = 100
K_test = 5

#Theta_P set
omega_test = np.array([8e-6,2e-6,4e-6,6e-6,8e-6])
alpha_test = np.array([0.04,0.03,0.02,0.03,0.04])
beta_test = np.array([0.7,0.08,0.75,0.8,0.7])
mu_test = np.array([-0.00005,-0.000002,0.000001,0.000003,0.000005])
gamma_test =  np.array([0.001,-0.002,0.003,-0.004,0.005])

#Theta_C set
Corr_test = np.array([[1.00,-0.10,0.12,-0.14,0.16],
                 [-0.10,1.00,0.18,-0.20,0.22],
                 [0.12,0.18,1.00,0.24,-0.26],
                 [-0.14,-0.20,0.24,1.00,0.28],
                 [0.16,0.22,-0.26,0.28,1.00]])

#Theta_D set
dist_test = "NIG" #Choose between "MALap" , "NIG" , "MAt"

if (dist_test == "MALap"):
  G_lda_test = 4.25
  G_xi_test = 10e-10
  G_psi_test = 2 
elif (dist_test == "NIG"):
  G_lda_test = -0.5 
  G_xi_test = 4.25
  G_psi_test = 1 
elif (dist_test == "MAt"):
  G_xi_test = 4.25
  G_lda_test = -0.5 * G_xi_test
  G_psi_test = 10e-10

#Simulated returns
G_test = create_G(dist_test,G_lda_test,G_xi_test,G_psi_test,T_test)
Z_test = sc.norm.rvs(size=T_test*K_test).reshape(K_test,T_test)
y_test = create_y(omega_test,alpha_test,beta_test,mu_test,gamma_test,Corr_test,T_test,K_test,G_test,Z_test)

In [0]:
##Parameter estimation by simulated data##

#Set random seed
np.random.seed(9999)

#Initialization of parameters sets#
T = 100
K = 5

#Theta_P set
omega_guess = sc.uniform.rvs(scale = 10e-5, size=K)
alpha_guess = sc.uniform.rvs(scale = 0.05, size=K)
beta_guess = sc.uniform.rvs(loc=0.5,scale=0.5, size=K)
mu_guess = sc.uniform.rvs(scale = 10e-5, size=K)
gamma_guess = sc.uniform.rvs(loc=-0.0001,scale=0.0071, size=K)
print("Initial value for parameter set Theta_P:\n {}\n".format(np.array([omega_guess,alpha_guess,beta_guess,mu_guess,gamma_guess])))

#Theta_C set
Corr_vec = sc.uniform.rvs(loc=-0.2,scale=0.4,size=int(0.5*K*(K-1)))
Corr_guess = CM_1_C_para_reshape(Corr_vec)
print("Initial value for parameter set Theta_C:\n {}\n".format(Corr_guess))

#Theta_D set
dist = "NIG" #Choose between "MALap" , "NIG" , "MAt"

if (dist == "MALap"):
  G_lda_guess = sc.uniform.rvs(loc=1,scale=1,size=1)
  G_xi_guess = 10e-10
  G_psi_guess = 2 
  print("Initial value for parameter set Theta_D:\n {}\n".format(np.array([G_lda_guess.item(),G_xi_guess,G_psi_guess])))
elif (dist == "NIG"):
  G_lda_guess = -0.5 
  G_xi_guess =  sc.uniform.rvs(loc=1,scale=1,size=1)
  G_psi_guess = 1 
  print("Initial value for parameter set Theta_D:\n {}\n".format(np.array([G_lda_guess,G_xi_guess.item(),G_psi_guess])))
elif (dist == "MAt"):
  G_xi_guess = sc.uniform.rvs(loc=1,scale=1,size=1)
  G_lda_guess = -0.5 * G_xi_guess
  G_psi_guess = 10e-10
  print("Initial value for parameter set Theta_D:\n {}\n".format(np.array([G_lda_guess,G_xi_guess,G_psi_guess.item()])))

#parameters for EM algo
iteration_num = 1000
opt_method = 'L-BFGS-B'
tol = 10e-5

#Parameter estimation
omega_final,alpha_final,beta_final,mu_final,gamma_final,Corr_final,G_lda_final,G_xi_final,G_psi_final,_ = EM_algo(omega_guess,alpha_guess,beta_guess,mu_guess,gamma_guess,Corr_guess,dist,G_lda_guess,G_xi_guess,G_psi_guess,T,K,y_test,iteration_num,opt_method,tol)

In [0]:
##Read files##

import pandas as pd
import os

all_files = os.listdir()

#cnt = 0
#for filename in all_files:
  #if filename.endswith(".csv"):
    #cnt += 1

close = []

cnt = 0
for filename in all_files:
  if filename.endswith(".csv"):
    cnt += 1
    data = pd.read_csv(filename)
    data_close = np.array(data["close"])
    close.append(data_close)

In [0]:
##Parameter estimation by real data##

np.random.seed(1357)
price = np.array(close).reshape(len(close[0]),len(close))
price = price.T
returns = np.divide(price[:,1:1259] - price[:,0:1258],price[:,0:1258])

T = returns.shape[1]
K = returns.shape[0]

garch_para = pd.read_csv("Estimation_param.csv")
omega_init = np.array(garch_para["V1"])
alpha_init = np.array(garch_para["V2"])
beta_init = np.array(garch_para["V3"])
mu_init = np.array(pd.read_csv("Estimation_mu.csv"))
gamma_init = sc.uniform.rvs(loc=-0.0001,scale=0.0071, size=K)

Corr_init = np.array(pd.read_csv("Estimation_Gam.csv"))

dist = "NIG"
lda_init = -0.5
xi_init = sc.uniform.rvs(loc=1,scale=2,size=1)
psi_init = 1

#parameters for EM algo
iteration_num = 100
opt_method = 'L-BFGS-B'
tol = 10e-5

#Parameter estimation
omega_final,alpha_final,beta_final,mu_final,gamma_final,Corr_final,G_lda_final,G_xi_final,G_psi_final,_ = EM_algo(omega_init,alpha_init,beta_init,mu_init,gamma_init,Corr_init,dist,lda_init,xi_init,psi_init,T,K,returns,iteration_num,opt_method,tol)

In [0]:
#Save parameters into csv

np.savetxt("omega.csv", omega_final, delimiter=",")
np.savetxt("alpha.csv", alpha_final, delimiter=",")
np.savetxt("beta.csv", beta_final, delimiter=",")
np.savetxt("mu.csv", mu_final, delimiter=",")
np.savetxt("gamma.csv", gamma_final, delimiter=",")
np.savetxt("Corr.csv", Corr_final, delimiter=",")
np.savetxt("G_para.csv", np.array([G_lda_final,G_xi_final,G_psi_final]), delimiter=",")